In [1]:
from pathlib import Path
from statistics import mean
from collections import defaultdict
import glob
from itertools import product

import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import pandas as pd
import numpy as np

import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio

# px.defaults.template = 'ggplot2'
px.defaults.template = 'ggplot2'
# pio.templates.default = 'simple_white'

scoring_functions = ['evoef2', 'bluues', 'piepisa', 'gmxmmpbsa']
pd.options.plotting.backend = "plotly"
home_dir = Path("..").resolve()
rtdos = home_dir / "rtdos_log"
d11_unres = Path(home_dir, "run", "optimization", "d11fr0", "work_dir")
d11_res = Path(home_dir, "run", "restrained_optimization", "d11", "work_dir")

# d11

In [2]:
def read_from_wrk_dir(dir: Path, SFs):

    its = list(filter(lambda x: "bu" not in x, glob.glob(str(dir / "*-B_*"))))
    its = sorted(its, key=lambda x: int(x.split('/')[-1].split('-')[0]))
    ites_d = dict.fromkeys([ Path(it).name for it in its ])

    for it in its:
        it_scores = {}
        for SF in SFs:
            try:
                scores_fn = Path(it, "scoring", f"scores_{SF}")
                with open(scores_fn, "r") as f:
                    score = [float(linea.strip()) for linea in f]
                    avg_score = mean(score)
                    it_scores[SF] = avg_score
            except FileNotFoundError as e:
                # raise FileNotFoundError from e
                print(scores_fn)
                print(f"{it} doesn't have scores.")
                continue
        ites_d[Path(it).name] = it_scores
    return pd.DataFrame(ites_d).T

In [3]:
res_scores = read_from_wrk_dir(Path(d11_res), scoring_functions)
unres_scores = read_from_wrk_dir(Path(d11_unres), scoring_functions)

In [4]:
with open(Path(d11_res, "top_iterations.csv"), 'r') as f:
    res_top_iters = [ linea.strip() for linea in f]

with open(Path(d11_unres, "top_iterations.csv"), 'r') as f:
    unres_top_iters = [ linea.strip() for linea in f]

In [5]:
scores = pd.concat([res_scores, unres_scores])
# x_epochs = np.array(scores.index.astype("string").str.split('-', expand=True).droplevel([1, 2, 3]).astype('int'))
x = np.concatenate([[0], np.repeat(np.arange(1, 11), 4, axis=0), [11],
                    np.repeat(np.arange(12, 53), 4, axis=0)])
x = [ int(iter[0].split('-')[0]) for iter in scores.iterrows() ]
x_topunres = [ int(iter.split('-')[0]) for iter in unres_top_iters ]
x_topres = [ int(iter.split('-')[0]) for iter in res_top_iters ]

# Had to do this x index by hand:
# x_topunres_0w = np.array([12, 12, 12, 13, 14, 14, 14, 15, 15, 16, 17, 17, 18, 19, 19, 20, 21, 22, 23, 24, 24, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 33])
# x_topunres_1w = np.array([34, 35, 36, 36, 36])
# x_topunres_2w = np.array([37, 38, 39, 40, 40, 41, 42, 42])
# x_topunres_3w = np.array([43, 43, 44, 44, 45, 45, 46, 47, 48, 49, 50, 50, 50, 50, 51, 51, 52, 52])
# x_topunres = np.concatenate([x_topunres_0w, x_topunres_1w, x_topunres_2w, x_topunres_3w])

In [37]:
fig = make_subplots(rows=1, cols=4)
colors = {SF: col for SF, col in zip(scoring_functions,px.colors.qualitative.Plotly )}

for SF, (row, col) in zip(scoring_functions, list(product([1], [1, 2, 3, 4]))):
    fig.add_trace(go.Scatter(x = x, y=scores[SF], mode='markers', name=SF,
                             marker_color="lightgrey", showlegend=False),
                  row = row, col = col)
    fig.add_trace(go.Scatter(x = x_topres, y=scores.loc[res_top_iters][SF], mode='markers', name=SF, marker_color=colors[SF], showlegend=False), row = row, col = col)
    fig.add_trace(go.Scatter(x = x_topunres, y=scores.loc[unres_top_iters][SF], mode='markers', name=SF, marker_color=colors[SF]), row = row, col = col)

    fig.update_xaxes(tickmode='linear', tick0=0, dtick=5, row=row, col=col)


fig.update_yaxes(title_text="score", row=1, col=1)
fig.update_xaxes(title_text="epoch", row=1, col=1)
fig.update_xaxes(title_text="epoch", row=1, col=2)
fig.update_xaxes(title_text="epoch", row=1, col=3)
fig.update_xaxes(title_text="epoch", row=1, col=4)

fig.add_vline(x = 6, line_width=1, line_dash='dash');

In [38]:
fig.update_layout(height = 600, width = 1200, font={'size': 20},
                      legend_title="Scoring Function", paper_bgcolor='rgba(0,0,0,0)')

In [39]:
fig.write_image(Path(rtdos, "d11_scores.png"))